In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

# Initialize client
client = bigquery.Client(project='rock-finder-project')

In [ ]:
import pandas_gbq

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sql = "SELECT * FROM rock-finder-project.routes.mp_routes"
p_id="rock-finder-project"
df_routes = pandas_gbq.read_gbq(sql, project_id=p_id)
df_routes


In [ ]:
#Null value in Rating
df_routes[['Rating']].isnull().sum()

In [ ]:
# How many 'Alpine' in Route Type
df_routes['Route Type'].value_counts()

#making the simple columns with only Sport, Trap, TR from 'Route Type' column

# Sample structure from your list
data =['Route Type']
df = pd.DataFrame(df_routes)
df






In [ ]:
# Define high-level route categories to track
core_types = ['Trad', 'Sport', 'TR']

In [ ]:
# Create columns for counting the number of each type
for route in core_types:
    df[f'is_{route.lower()}'] = df['Route Type'].str.contains(rf'\b{route}\b').astype(int)

In [ ]:
df

# the number of each column
df[['is_trad', 'is_sport', 'is_tr']].sum()

In [ ]:
# day 2
# cleaning URL columns
df['url'].isnull().sum()

In [ ]:
df['url'].info()


In [ ]:
df['url'].duplicated().sum()

In [ ]:
# check url validity -> ensuring the data contains real, usable links 
import re



In [ ]:
def is_valid_url(url):
    return bool(re.match(r'^https?://', str(url).strip()))

In [ ]:
print(df['url'].apply(is_valid_url).value_counts())
#all true, so far so good

In [ ]:
# cleaning avg_stars
df['avg_stars'].isnull().sum()

In [ ]:
df['avg_stars'].info()

In [ ]:
df['avg_stars'].describe()

In [ ]:
#skew is not bad
df['avg_stars'].skew()

In [ ]:
# checking outliers
import seaborn as sns

In [ ]:
sns.boxplot(x=df['avg_stars'])
# we can see a few of dots on the left side but it doesn't seem like outliers

In [ ]:
#in [' num_votes'] column, erase blank
df.rename(columns={" num_votes": "num_votes"}, inplace=True)

In [ ]:
#starting cleaning num_votes column
df['num_votes'].isnull().sum()

In [ ]:
df['num_votes'].max()

In [ ]:
df['num_votes'].min()

In [ ]:
#too much gap between max and min so let's investigate median and skew first
df['num_votes'].describe()

In [ ]:
#check the skew
df['num_votes'].skew()

In [ ]:
import seaborn as sns
sns.boxplot(x=df['num_votes'])

In [ ]:
df['num_votes'].sort_values(ascending=False).head(20)

In [ ]:
# drawing graph
df[['num_votes']].sort_values(by='num_votes').reset_index(drop=True).plot()

In [ ]:
#highly right skewed so we need to do something for getting rid of outliers
#or we can find more information of whole data rows of huge amount of 'num_votes' not to make wrong decision

In [ ]:
# 1. the list of massive values from num_votes that we want to find 
high_votes = [1757, 1379, 1323, 1284, 1159, 1156, 1080, 1066, 1044, 1020, 990
, 989, 959, 957, 925, 917, 907, 868, 861, 844]

In [ ]:
# 2. extracting the whole range of rows align with this condition/ check the url for gathering the info
subset_df = df[df['num_votes'].isin(high_votes)]

In [ ]:
# *get a full url
pd.set_option('display.max_colwidth', None)

In [ ]:
# 3. arrangement + without desc column
subset_df = subset_df.sort_values(by='num_votes', ascending=False)
subset_df = subset_df.drop(columns=[' desc'])
subset_df 

In [ ]:
#after checking information in url, we can decide it seems to be wrong to extract the huge values in num_votes.